In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# Load the dataset
df = pd.read_csv('shortened_oil_data.csv')

# 1. Preprocessing
# Convert 'Date' to datetime and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Impute missing values using forward fill (common for time-series data)
df_imputed = df.fillna(method='ffill')

# Drop any remaining NaNs (e.g., NaNs at the very start of the time series)
df_imputed.dropna(inplace=True)

# Define the NEW Target
target = 'Crude_Oil'
y = df_imputed[target]

# Baseline Features (All numerical columns except the new target)
numerical_cols = df_imputed.select_dtypes(include=np.number).columns
# Filter out the target column from the feature list
baseline_features = [col for col in numerical_cols if col != target]
X_baseline = df_imputed[baseline_features]


# 2. Data Splitting (using 80/20 split, shuffle=False for time-series)
test_size = 0.2
random_state = 42

# Baseline Split
X_bl_train, X_bl_test, y_bl_train, y_bl_test = train_test_split(
    X_baseline, y, test_size=test_size, random_state=random_state, shuffle=False
)

# Feature Selected Split
#X_fs_train, X_fs_test, y_fs_train, y_fs_test = train_test_split(
#    X_feature_selected, y, test_size=test_size, random_state=random_state, shuffle=False
#)

X_fs_train = X_bl_train.filter(regex='PADD3|PADD3_RefBl_NetProd_FinGas|Residual|US_Crude_Stocks_Transit_from_AK', axis=1)
X_fs_test = X_bl_test.filter(regex='PADD3|PADD3_RefBl_NetProd_FinGas|Residual|US_Crude_Stocks_Transit_from_AK', axis=1)
y_fs_train = y_bl_train
y_fs_test = y_bl_test

# 3. Random Forest Model Implementation and Evaluation

# Initialize Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=500, random_state=random_state)
grid_values = {'max_features': np.linspace(1,100,100, dtype='int32'),
               'min_samples_leaf': [5],
               'n_estimators': [500],
               'random_state': [42]}
cv = KFold(n_splits=5,random_state=42,shuffle=True)
rf_bl = rf_model
rf_cv = GridSearchCV(rf_bl, param_grid=grid_values, scoring='accuracy', cv=cv,verbose=2)

# --- Model 1: Baseline Features ---
rf_cv.fit(X_bl_train, y_bl_train)
y_bl_pred = rf_cv.predict(X_bl_test)

# Evaluation Metrics for Baseline
mse_bl = mean_squared_error(y_bl_test, y_bl_pred)
r2_bl = r2_score(y_bl_test, y_bl_pred)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[CV] END max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88; total time=   0.3s


/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[CV] END max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88; total time=   0.3s


...

[CV] END max_features=100, min_samples_leaf=5, n_estimators=500, random_state=88; total time=   6.7s


/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[CV] END max_features=100, min_samples_leaf=5, n_estimators=500, random_state=88; total time=   6.4s


/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wwang/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[CV] END max_features=100, min_samples_leaf=5, n_estimators=500, random_state=88; total time=   6.5s


In [11]:
# --- Model 2: Feature Selected Features ---
rf_fs = rf_model
rf_fs.fit(X_fs_train, y_fs_train)
y_fs_pred = rf_fs.predict(X_fs_test)

# Evaluation Metrics for Feature Selected
mse_fs = mean_squared_error(y_fs_test, y_fs_pred)
r2_fs = r2_score(y_fs_test, y_fs_pred)

# 4. Results Compilation
results = pd.DataFrame({
    'Model': ['Baseline', 'Feature Selected'],
    'MSE': [mse_bl, mse_fs],
    'R-squared': [r2_bl, r2_fs]
})

print(results)

              Model         MSE  R-squared
0          Baseline  168.006357  -5.717516
1  Feature Selected   82.994200  -2.318415


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# Load the dataset
df = pd.read_csv('shortened_oil_data.csv')

# 1. Preprocessing
# Convert 'Date' to datetime and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Impute missing values using forward fill (common for time-series data)
df_imputed = df.fillna(method='ffill')

# Drop any remaining NaNs (e.g., NaNs at the very start of the time series)
df_imputed.dropna(inplace=True)

# Define the NEW Target
target = 'Crude_Oil'
y = df_imputed[target]

# Baseline Features (All numerical columns except the new target)
numerical_cols = df_imputed.select_dtypes(include=np.number).columns
# Filter out the target column from the feature list
baseline_features = [col for col in numerical_cols if col != target]
X_baseline = df_imputed[baseline_features]


# 2. Data Splitting (using 80/20 split, shuffle=False for time-series)
test_size = 0.2
random_state = 42

# Baseline Split
X_bl_train, X_bl_test, y_bl_train, y_bl_test = train_test_split(
    X_baseline, y, test_size=test_size, random_state=random_state, shuffle=False
)

# Feature Selected Split
#X_fs_train, X_fs_test, y_fs_train, y_fs_test = train_test_split(
#    X_feature_selected, y, test_size=test_size, random_state=random_state, shuffle=False
#)

X_fs_train = X_bl_train.filter(regex='PADD3|PADD3_RefBl_NetProd_FinGas|Residual|US_Crude_Stocks_Transit_from_AK', axis=1)
X_fs_test = X_bl_test.filter(regex='PADD3|PADD3_RefBl_NetProd_FinGas|Residual|US_Crude_Stocks_Transit_from_AK', axis=1)
y_fs_train = y_bl_train
y_fs_test = y_bl_test

# 3. Random Forest Model Implementation and Evaluation

# Initialize Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=500, random_state=random_state)
grid_values = {'max_features': np.linspace(1,100,100, dtype='int32'),
               'min_samples_leaf': [5],
               'n_estimators': [500],
               'random_state': [42]}
cv = KFold(n_splits=5,random_state=42,shuffle=True)
rf_bl = rf_model
rf_cv = GridSearchCV(rf_bl, param_grid=grid_values, scoring='r2', cv=cv,verbose=2)

# --- Model 1: Baseline Features ---
rf_cv.fit(X_bl_train, y_bl_train)
y_bl_pred = rf_cv.predict(X_bl_test)

# Evaluation Metrics for Baseline
mse_bl = mean_squared_error(y_bl_test, y_bl_pred)
r2_bl = r2_score(y_bl_test, y_bl_pred)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88; total time=   0.4s
[CV] END max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88; total time=   0.3s
[CV] END max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88; total time=   0.3s
[CV] END max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88; total time=   0.3s
[CV] END max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88; total time=   0.3s
[CV] END max_features=2, min_samples_leaf=5, n_estimators=500, random_state=88; total time=   0.3s
[CV] END max_features=2, min_samples_leaf=5, n_estimators=500, random_state=88; total time=   0.3s
[CV] END max_features=2, min_samples_leaf=5, n_estimators=500, random_state=88; total time=   0.3s
[CV] END max_features=2, min_samples_leaf=5, n_estimators=500, random_state=88; total time=   0.3s
[CV] END max_features=2, min_samples_leaf=5, n

In [13]:
# --- Model 2: Feature Selected Features ---
rf_fs = rf_model
rf_fs.fit(X_fs_train, y_fs_train)
y_fs_pred = rf_fs.predict(X_fs_test)

# Evaluation Metrics for Feature Selected
mse_fs = mean_squared_error(y_fs_test, y_fs_pred)
r2_fs = r2_score(y_fs_test, y_fs_pred)

# 4. Results Compilation
results = pd.DataFrame({
    'Model': ['Baseline', 'Feature Selected'],
    'MSE': [mse_bl, mse_fs],
    'R-squared': [r2_bl, r2_fs]
})

print(results)

              Model        MSE  R-squared
0          Baseline  10.491946   0.580493
1  Feature Selected  82.994200  -2.318415


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
df = pd.read_csv('shortened_oil_data.csv')

# 1. Preprocessing
# Convert 'Date' to datetime and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Impute missing values using forward fill (common for time-series data)
df_imputed = df.fillna(method='ffill')

# Drop any remaining NaNs (e.g., NaNs at the very start of the time series)
df_imputed.dropna(inplace=True)

# Define the NEW Target
target = 'Crude_Oil'
y = df_imputed[target]

# Baseline Features (All numerical columns except the new target)
numerical_cols = df_imputed.select_dtypes(include=np.number).columns
# Filter out the target column from the feature list
baseline_features = [col for col in numerical_cols if col != target]
X_baseline = df_imputed[baseline_features]


# 2. Data Splitting (using 80/20 split, shuffle=False for time-series)
test_size = 0.2
random_state = 42

# Baseline Split
X_bl_train, X_bl_test, y_bl_train, y_bl_test = train_test_split(
    X_baseline, y, test_size=test_size, random_state=random_state, shuffle=False
)

X_fs_train = X_bl_train.filter(regex='PADD3|PADD3_RefBl_NetProd_FinGas|Residual|US_Crude_Stocks_Transit_from_AK', axis=1)
X_fs_test = X_bl_test.filter(regex='PADD3|PADD3_RefBl_NetProd_FinGas|Residual|US_Crude_Stocks_Transit_from_AK', axis=1)
y_fs_train = y_bl_train
y_fs_test = y_bl_test

# 3. Random Forest Model Implementation and Evaluation

# Initialize Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=500, random_state=random_state, n_jobs=-1)

# --- Model 1: Baseline Features ---
rf_bl = rf_model
rf_bl.fit(X_bl_train, y_bl_train)
y_bl_pred = rf_bl.predict(X_bl_test)

# Evaluation Metrics for Baseline
mse_bl = mean_squared_error(y_bl_test, y_bl_pred)
r2_bl = r2_score(y_bl_test, y_bl_pred)

# --- Model 2: Feature Selected Features ---
rf_fs = rf_model
rf_fs.fit(X_fs_train, y_fs_train)
y_fs_pred = rf_fs.predict(X_fs_test)

# Evaluation Metrics for Feature Selected
mse_fs = mean_squared_error(y_fs_test, y_fs_pred)
r2_fs = r2_score(y_fs_test, y_fs_pred)

# 4. Results Compilation
results = pd.DataFrame({
    'Model': ['Baseline', 'Feature Selected'],
    'MSE': [mse_bl, mse_fs],
    'R-squared': [r2_bl, r2_fs]
})

print(results)

              Model        MSE  R-squared
0          Baseline   1.631507   0.934766
1  Feature Selected  82.994200  -2.318415
